## Raw Data Extraction

In this section we manualy insert the the key core data which consist of symbol, entry/exit time and date, entry price, exit price, and some additional tags. Based on the symbol, date and time keys we download additional 1 minute,1 hour and 1 day bar data and fundamental data for each symbol and date inserted.

Sources:

* 1 minute bar data: Interactive Brokers and Yahoo Finance.
* 1 hour bar data: Yahoo Finance.
* 1 day bar data: Yahoo Finance.
* Fundamentals data: Yahoo Finace.

The data from Yahoo Finance will be downloaded using the yfinance library and the data from Interactive Brokers will be downloaded using the Interactive Brokers API (only possible with a valid IB account and a monthly data subscrition).

The trade picking process is carried out in a discretionary manner (manualy) using the TWS Interactive Brokers trading platform, which brings us to reason why the 1 minute data is downloaded from Interactive Brokers. Yahoo Finance 1 minute data will be used only if necessary depending on the accuracy and availability of the Interactive Brokers data, in other words 1 minutes Yahoo Finance data is used for backup.

Since data from Yahoo Finance is relatively accurate, reliable and easily downloaded it is sufficent for the 1 hour and 1 day bar data which is why data of these time periods from Interactive Brokers is unnecessary. 


In [1]:
# Import libraries
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
import threading
import datetime
import time
import os #kill the kernel
import sys
import yfinance as yf
import appfunctions as ap
import numpy as np




### Trades Snap-Shot Entry (daily manual data entry)

Here we enter manually each trade's properties, this is core information of each trade.\
Each trade is inserted individually. 


In [64]:


##### symbol #####
symbol = 'APRE'

##### intended entry #####
intended_entry = 5.73

##### SL #####
SL = 5.5

##### Generic Exit #####
exit =  5.5

##### Entry Time #####
entry_time = datetime.datetime(2021,7,21,10,20)

#### General Exit Time #####
#NOTE: if exited not on SL then always write the following candle after the exit signal candle
exit_time = datetime.datetime(2021,7,21,10,40)

#### VWAP TAG ##### 
#  BO,  SUPPORT,  FALSE
vwap_tag="SUPPORT"

#### Pattern #####
# W- wedge, F - flag, AT -ascending triangle, DT - descending triangle, ST - symetrical triangle, R -  rectangle, P- penannt
pattern ='ST'

#### Catalyst #####
# H  - hype, L - leading industry/sector, C - news catalyst
catalyst = "C"


#### Strategy ####
strategy ="BO"

#### Entry ####
entry = intended_entry
###############################################


### Check Errors

This cell validates the manual data entrys to avoid errors and future cleaning.
Makes sure dates, time and prices are aligned.


In [65]:
# errors:
if SL>=intended_entry:
    print("******** ERROR: SL higher than entry ********")
    sys.exit()
    

if (entry_time>exit_time) or (entry_time.date()!=exit_time.date()):
    print("******** ERROR: Incorrect entry time or exit time  ********")
    sys.exit()
    
day_start = datetime.datetime.combine(entry_time.date(),datetime.time(9,30))
day_end = datetime.datetime.combine(entry_time.date(),datetime.time(16,0))
    
if (entry_time<day_start or entry_time>=day_end) or (exit_time<day_start or exit_time>=day_end):
    print("******** ERROR: Invalid entry time or exit time, exceeds the trading hours range  ********")
    sys.exit()



### Store The Core and Fundamentals Data

Based on given symbol and date, download the fundamental data of that given symbol and date and store it in the Fundamentals dataset for future analysis and prediciton. Locally store the core data as insterted above and the freshly downloaded fundamental data alongside the data previously added.

In [66]:
# Create the new core data row and core keys.

core_key = {"Symbol":symbol,"Date": entry_time.date()}
core_dict = {"Entry Time": entry_time.time(),"Exit Time": exit_time.time(), "Intended Entry":intended_entry,
"Entry": entry, "SL": SL,"Exit": exit,"Pattern": pattern,"VWAP Tag": vwap_tag,"Strategy":strategy,"Catalyst":catalyst,"Download":0}

# import the datasets 
core_data =  pd.read_excel('Core Data.xlsx')  
f_data =  pd.read_excel('Fundamentals.xlsx')

# a function that merges to dictionaries
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

# timestamp - Datetime type changes
timestamp_date = pd.Timestamp(entry_time.date())
strftime_time = entry_time.time().strftime("%H:%M:%S")

# add the Core Data dictionary to the Core Data data set + a warning if the data exists to avoid duplicates
if len(core_data)==0 or len(core_data[(core_data["Symbol"]==symbol) & (core_data["Entry Time"]==strftime_time)
& (core_data["Date"]==timestamp_date)])==0:    
    core_dict = merge_two_dicts(core_key, core_dict)
    core_data =  core_data.append(core_dict, ignore_index=True)
    core_data.to_excel("Core Data.xlsx",index = False) 
else:
    print("******** WARNING: Given key already inserted   ********")

# download and add the Fundamentals dictionary to the Fundamentals data set 
if len(f_data)==0 or (len(f_data[(f_data["Symbol"]==symbol) & (f_data["Date"]==timestamp_date)])==0):
    ticker = yf.Ticker(symbol)
    fundamentals= ticker.info
    fundamentals= merge_two_dicts(core_key, fundamentals)
    f_data =  f_data.append(fundamentals, ignore_index=True)
    f_data.to_excel("Fundamentals.xlsx",index = False)  


### Intra-day Data collection

Different function for utilizing the data collection from the IB API.

In [2]:



class TradeApp(EWrapper, EClient): 
    def __init__(self, addr, port, client_id): 
        
        EWrapper.__init__(self)
        EClient.__init__(self, self)
        
        self.connect(addr, port, client_id)
        self.data = {}
        thread = threading.Thread(target=self.run)
        thread.start()
        
    def historicalData(self, reqId, bar):
        if reqId not in self.data:
            self.data[reqId] = [{"Datetime":bar.date,"Open":bar.open,"High":bar.high,"Low":bar.low,"Close":bar.close,"Volume":bar.volume}]
        else:
            self.data[reqId].append({"Datetime":bar.date,"Open":bar.open,"High":bar.high,"Low":bar.low,"Close":bar.close,"Volume":bar.volume})
        #print("reqID:{}, date:{}, open:{}, high:{}, low:{}, close:{}, volume:{}".format(reqId,bar.date,bar.open,bar.high,bar.low,bar.close,bar.volume))

def usTechStk(symbol,sec_type="STK",currency="USD",exchange="ISLAND"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.currency = currency
    contract.exchange = exchange
    return contract 

def histData(req_num,contract,endDate,duration,candle_size):
    """extracts historical data"""
    app.reqHistoricalData(reqId=req_num, 
                          contract=contract,
                          endDateTime=endDate,
                          durationStr=duration,
                          barSizeSetting=candle_size,
                          whatToShow='TRADES',
                          useRTH=1,
                          formatDate=1,
                          keepUpToDate=0,
                          chartOptions=[])	 # EClient function to request contract details    

# storing trade app object data (the downloaded data) in a dataframe 
def dataDataframe(symbols,TradeApp_obj):
    "returns extracted historical data in dataframe format"
    df_data = {}
    i=0
    for symbol in symbols:
        df_data[symbol] = pd.DataFrame(TradeApp_obj.data[i])
        df_data[symbol].set_index("Datetime",inplace=True)
        i+=1
    return df_data

# convert string to datetime and validate the time
def datetimeCon(x):
    date_time_obj = datetime.datetime.strptime(x, '%Y%m%d %H:%M:%S')
    if date_time_obj.time()< datetime.time(9,30) or date_time_obj.time()>datetime.time(16,0):
        print("******** WARNING: Unsual time, change IB time-zone ********")
        print(date_time_obj.time())
        sys.exit()
    return date_time_obj



### Download Raw Data

In this cell we will download the bar data from the soreces as mentioned abov.

In [15]:
#import the Core Data
core_data =  pd.read_excel('Core Data.xlsx') 

# Filter the data to get only the rows where data has not yet been downloaded (where the column Download is 0)
download_data = core_data[core_data["Download"]==0][["Symbol","Date"]].drop_duplicates().reset_index()
print(download_data)


# connect to IB server
app = TradeApp('127.0.0.1', 7497,6)
#app.connect(host='127.0.0.1', port=7497, clientId=2) #port 4002 for ib gateway paper trading/7497 for TWS paper trading
#con_thread = threading.Thread(target=websocket_con, daemon=True)
#time.sleep(1)
#con_thread.start()
time.sleep(1) # some latency added to ensure that the connection is established


# make an API call to download the 1 minute data from IB
for index,row in download_data.iterrows():    
    date = datetime.datetime.combine(row["Date"].date(),datetime.time(23,59)).strftime("%Y%m%d %H:%M:%S")
    
    print("check 1.11")
    symbol = row["Symbol"] 
    dict_download = {"Symbol": symbol,"Date":date}
    
    print(str(symbol)+ " " + date)
    histData(index,usTechStk(symbol),date,'1 D', '1 min')
    time.sleep(5)

        



#extract and store IB 1 minute data in dataframes
historicalData = dataDataframe(download_data["Symbol"].values,app)

# store the data localy
# store intraday IB data from the broker

    
# store intraday and D1 data from Yahoo finance, IB and merge the Yahoo volume only with the IB data 

#the local directories for the bar data
directory1 = 'Yahoo Intraday Data'
directory2 = 'Yahoo D1 Data'
directory3 = 'Yahoo H1 Data'
directory4 = 'IB Intraday Data'
directory5 = 'Merged Intraday Data'
directory6 = 'SPY Intraday Data'
directory7 = 'SPY D1 Data'

time_period1 = 365 # 1 year
time_period2 = 30 #1 month

# for every symbol download the intrady,1h and 1d bar data
for key in historicalData:
    print(key)

    
    date_timestamp = historicalData[key].index[0].split()[0]
    date = datetime.datetime.strptime(date_timestamp, '%Y%m%d').date()

    # download the 1 minute data from Yahoo Finance
    minute_data_yahoo =  yf.download(key, start = date, end =  (date + datetime.timedelta(days=1)), interval = "1m" )  

    # Remove time zone from date-time index value
    minute_data_yahoo.index = minute_data_yahoo.index.map(lambda x: datetime.datetime.combine(x.date(),x.time()).strftime("%Y%m%d  %H:%M:%S"))

    # download the d1 and 1h bar data from Yahoo Finance
    d1_data=  yf.download(key,  end =  (date + datetime.timedelta(days=1)), interval = "1d" )
    h1_data=  yf.download(key, start = (date - datetime.timedelta(days=time_period2)), end =  (date + datetime.timedelta(days=1)), interval = "1h" )
    h1_data.index = h1_data.index.map(lambda x: datetime.datetime.combine(x.date(),x.time()).strftime("%Y%m%d  %H:%M:%S"))

    #create strings for file names
    file_name = key + ' ' + date_timestamp + '.xlsx'
    
    spy_data_file_name = "SPY "+date_timestamp + ".xlsx"
    
    spy_files =os.listdir(directory6)
    

    
    if spy_data_file_name not in spy_files:
            spy_minute_data_yahoo =  yf.download("SPY", start = date, end =  (date + datetime.timedelta(days=1)), interval = "1m" )
            spy_minute_data_yahoo.index = spy_minute_data_yahoo.index.map(lambda x: datetime.datetime.combine(x.date(),x.time()).strftime("%Y%m%d  %H:%M:%S"))
            directory_destination = "SPY Intraday Data"
            
            

            path6 = os.path.join(directory6, spy_data_file_name)
            spy_minute_data_yahoo.to_excel(path6)

        


    # merge the data so that high,low, open, close are taken from IB and Volume is taken from Yahoo
    merged_data = pd.merge(historicalData[key].drop(columns = ["Volume"]),minute_data_yahoo[["Volume"]],on='Datetime',how='left')
    merged_data["Volume"] =merged_data["Volume"].fillna(0) 
    
    # every session is 390 minutes starting at 9:30 and ending at the end of 15:59
    # leave a warning if some minutes are missing
    minutes_per_session = 390
    if len(merged_data)!=minutes_per_session:
        print("******** WARNING: The length is not 390, please fix the lenth of symbol" + key +" ********")
        
    

    #local paths to the directories
    path1 = os.path.join(directory1, file_name)
    path2 = os.path.join(directory2, file_name.split()[0] + " D1.xlsx")
    path3 = os.path.join(directory3, file_name)
    path4 = os.path.join(directory4, file_name)
    path5 = os.path.join(directory5, file_name)
    
    
    # save the datasets in excel files in the paths
    minute_data_yahoo.to_excel(path1)
    d1_data.to_excel(path2)
    h1_data.to_excel(path3)
    historicalData[key].to_excel(path4)
    merged_data.to_excel(path5)


# download daily spy data up to this day    
today = datetime.datetime.today().date()
spy_d1_data_yahoo =  yf.download("SPY", end  = today, interval = "1d" )
file_name  = 'SPY D1.xlsx'
path7 =   os.path.join(directory7, file_name)
spy_d1_data_yahoo.to_excel(path7) 

# update the core_data table, change download to 1 from 0 to signal that the data has been downloaded
core_data["Download"] =  core_data["Download"].replace({0:1})
core_data.to_excel('Core Data.xlsx',index = False) 






ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefeu


Empty DataFrame
Columns: [index, Symbol, Date]
Index: []
[*********************100%***********************]  1 of 1 completed


## Data Organizer and Extractor

Here we organize the data, extract relevant features and then store it for the main project.

### 3.2) Fundamentals


##### Raw:



|Term                  |Description                                                                                                                                                                                                                                                                                                                                                                                                                                                               |Format |
|:-----------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|
|Date                   |the date in which the data was downloaded                                                                                                                                                                                                                                                                                                                                                                                                                             |date   |
|Symbol                 |symbol of the stock                                                                                                                                                                                                                                                                                                                                                                                                                                                   |string |
|priceToBook            |a ratio that expresses a company's stock share value divided by its book value per share                                                                                                                                                                                                                                                                                                                                                                              |ratio  |
|country                |the country location of the company                                                                                                                                                                                                                                                                                                                                                                                                                                   |string |
|floatShares            |the portion of shares of a company that are in the hands of public investors                                                                                                                                                                                                                                                                                                                                                                                          |USD    |
|heldPercentInsiders    |the percentage of shares held by company insiders (board, management, those who are party to insider information)                                                                                                                                                                                                                                                                                                                                                     |percent|
|heldPercentInstitutions|the percentage of shares of a compnay held by institutions (mostly pension funds, insurance companies or investment banks)                                                                                                                                                                                                                                                                                                                                            |percent|
|marketCap              |the total value of all company's shares of stock. The number of shares multiplied by the value of one share of a stock                                                                                                                                                                                                                                                                                                                                              |USD    |
|netIncomeToCommon      |total net income to common stock holders. Calculated by: the total revenue of the company - total expenses - preferred dividends                                                                                                                                                                                                                                                                                                                                      |USD    |
|sector                 |the sector type of the stock                                                                                                                                                                                                                                                                                                                                                                                                                                          |string |
|sharesOutstanding      |the total number of shares of a company                                                                                                                                                                                                                                                                                                                                                                                                                               |int    |
|sharesShort            |the number of shares of a stock currently sold short and not yet closed or covered (returned to the lender)                                                                                                                                                                                                                                                                                                                                                           |int    |
|shortPercentOfFloat    |percentage of short sales of a stock                                                                                                                                                                                                                                                                                                                                                                                                                                  |percent|
|enterpriseToEbitda     |a ratio used as a valuation tool to compare the value of a company, debt included, to the company’s cash earnings less non-cash expenses.<br>enterprise is the overall value of a company, the sum of the marketcap + debts + value of associate companies - cash and cash equivalents.<br>EBITDA stands for Earnings Before Interest, Taxes, Depreciations and Amortizations, meaning the earnings of a company prior to subtracting expenses like interests or taxes.|ratio  |

In [16]:
fundamentals = pd.read_excel("Fundamentals.xlsx")

fundamentals = fundamentals[["Date","Symbol","priceToBook","country","floatShares","heldPercentInsiders","heldPercentInstitutions"
,"marketCap", "netIncomeToCommon","sector","sharesOutstanding","sharesShort","shortPercentOfFloat","enterpriseToEbitda"]]

fundamentals["Date"] = fundamentals["Date"].apply(lambda x: x.date())

### 3.3) Core Data

The Core data dataframe includes the data manually inserted for this project, each observation represents a trade and is added separately followed by the the trade's prices, times and tags presented in this table. The distinction of each trade can be seen looking at the data's keys: Symbol, Date, and Entry Time.


##### Raw:

|Term                 |Description                                                                                                                                                                                                                                                                                                                                                                                                                                                               |Format |
|:-----------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|
|Date                   |the date in which the trade took place                                                                                                                                                                                                                                                                                                                                                                                                                            |date   |
|Symbol                 |symbol of the stock                                                                                                                                                                                                                                                                                                                                                                                                                                                   |string |
|Entry            |the entry price of a given trade                                                                                                                                                                                                                                                                                                                                                                             |USD  |
|Entry Time                |the entry time of a given trade                                                                                                                                                                                                                                                                                                                                                                                                                                   |time | 
|SL            |stop loss price, if price exceeds this price to the downside sell immediately                                                                                                                                                                                                                                                                                                                                                                            |USD  |                          
|Exit            |the exit price of a given grade                                                                                                                                                                                                                                                                                                                                                                                          |USD    |
|Exit Time    |the exit time of a given trade                                                                                                                                                                                                                                                                                                                                                    |time|
|Pattern| the pattern tag (The correction pattern prior to breakout). See [1](#1)                                                                                                                                                                                                                                                                                                                                          |string|
|VWAP Tag              |the vwap tag. 'BO'-pattern breakout from  vwap or 'SUPPORT' - pattern supported by vwap. See [3.1](#3.1)                                                                                                                                                                                                                                                                                                                                          |string  |
|Catalyst      |a catalyst backing the trade. 'L' - leading sector/industry, 'H' - hype with no reason or 'C' - positive news catalyst                                                                                                                                                                                                                                                                                                                                  |string    |


##### Added Columns:


|Term                 |Description                                                                                                                                                                                                                                                                                                                                                                                                                                                               |Format |
|:-----------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|
|Num                   | index for each trade, for easier data joins instead of using the 3 keys   |int|    




Risk Reward Ratio:   $RRR = R = \frac {Exit \ Price \ - \ Entry \ Price}{Exit \ Price \ - \ Stop \  Loss}$.

If trade drops to stop loss $R = -1$, at breakeven  $R = 0$ and at profit $R > 0$.

|Term                 |Description                                                                                                                                                                                                                                                                                                                                                                                                                                       |Format |
|:-----------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|
|Exit R                   | risk reward ratio at exit  |int|  

In [17]:
core_data = pd.read_excel("Core Data.xlsx")
core_data = core_data[core_data["Download"]!=0]
core_data["Date"] = core_data["Date"].apply(lambda x: x.date())
core_data["Entry Time"] = core_data["Entry Time"].apply(lambda x: datetime.datetime.strptime(x ,'%H:%M:%S').time())
core_data["Exit Time"] = core_data["Exit Time"].apply(lambda x: datetime.datetime.strptime(x ,'%H:%M:%S').time())
core_data["Num"] = pd.Series(np.arange(1,len(core_data)+1))

core_data["Exit R"] = (core_data["Exit"]-core_data["Entry"])/((core_data["Entry"]-core_data["SL"]))

##### Organzie and extract data for the bar datasets.

In [18]:
%%time
sources = {"Yahoo Intraday Data":0,"IB Intraday Data":0,"Merged Intraday Data":0,"SPY Intraday Data":0,
           "Yahoo D1 Data":0,"SPY D1 Data":0}

core_path = os.getcwd()

i=0

for source in sources:
    path =  os.path.join(core_path,source)
    data = pd.DataFrame()
    for filename in os.listdir(path):


        file_path =  os.path.join(path , filename)

        df = pd.read_excel(file_path)
        symbol_name = filename.split()[0]
        df["Symbol"] = symbol_name
        min_per_session = 390
 
        if source == "Yahoo H1 Data" or source == "Yahoo D1 Data" or source== 'SPY D1 Data':

            # *********need to separate 1h periods!***********
            df["Date"] = df["Date"].apply(lambda x: x.date())

            
            # three months are 66 trading days (22 tracing days per month)
            three_months = 66
            df = ap.addRelativeVolume(df,period = three_months)
            
            df = ap.addGap(df)
        else:
            df["Datetime"] = df["Datetime"].apply(lambda x: datetime.datetime.strptime(x , '%Y%m%d  %H:%M:%S'))
            df["Date"] = df["Datetime"].apply(lambda x: x.date())
            df["Time"] = df["Datetime"].apply(lambda x: x.time())
            df = df.drop(columns = ["Datetime"])
            
            
            # apply the feature engineering function
            df = ap.addVwap(df)
            df = ap.addBarTag(df)
            df = ap.addRelativeVolume(df)

            df = ap.fillMissingBar(df)
            
        data = data.append(df,ignore_index = True)
        
    sources[source]=data

# The datasets after importing and changes
yahoo_minute =sources["Yahoo Intraday Data"]
IB_minute =sources["IB Intraday Data"]
merged = sources["Merged Intraday Data"]
spy_minute = sources["SPY Intraday Data"]      
                                        
#yahoo_h1 = sources["Yahoo H1 Data"]
yahoo_d1 = sources["Yahoo D1 Data"]

spy_d1 =sources["SPY D1 Data"] 

ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfuture


Wall time: 13min 35s


In [19]:
path = os.path.join('immidiate import', 'yahoo_minute.xlsx')
yahoo_minute.to_excel(path,index = 'Fasle')



path = os.path.join('immidiate import', 'spy_minute.xlsx')
spy_minute.to_excel(path,index = 'Fasle')


path = os.path.join('immidiate import', 'yahoo_d1.xlsx')
yahoo_d1.to_excel(path,index = 'Fasle')

path = os.path.join('immidiate import', 'spy_d1.xlsx')
spy_d1.to_excel(path,index = 'Fasle')

#### Intraday Data Pick - why we use only yahoo minute data from now onwards

The core data table is based on manualy inserted trades, where the decision of entering a specific trade is based using the TWS Intractive Brokers trading platform, which is why the IB minute data downloaded with the IB api was crucial. Suprisingly However, the data downloaded from IB with the api seems to be poor compared to the yahoo minute data. I have come to this conclusion after comparing the data sometimes bar by bar. 

The 'merged data' is a dataset with prices (open,high,close, low ) from IB and volume from Yahoo. The 'merged data' came to compensate for the poor volume data from IB because IB uses many filters to their volume data making it close to impossible to work with in terms of accuracy. 

This brings us to the conclusion that 'merged data' and 'IB minute data' will not play a part in this project for the reasons mentioned above.

#### Merge the minute data , spy minute data with the core data and compute the RRR - risk reward ratio for each bar

$RRR = R = \frac {Exit \ Price \ - \ Entry \ Price}{Exit \ Price \ - \ Stop \  Loss}$ 

In [20]:
# change the spy data columns names

selected_spy_minute = spy_minute.drop(columns  = ["Symbol","Adj Close","Date","Time"])

selected_spy_minute = selected_spy_minute.add_prefix('spy_')
selected_spy_minute[["Date","Time"]] = spy_minute[["Date","Time"]]

# compute the RRR for symbol minute data and merge it with the core data table
yahoo_minute_R = yahoo_minute.merge(core_data, on = [ "Date","Symbol"], how = "left")

yahoo_minute_R["High R"] =(yahoo_minute_R["High"]-yahoo_minute_R["Entry"])/(yahoo_minute_R["Entry"]-yahoo_minute_R["SL"])
yahoo_minute_R["Low R"] = (yahoo_minute_R["Low"]-yahoo_minute_R["Entry"])/(yahoo_minute_R["Entry"]-yahoo_minute_R["SL"])
yahoo_minute_R["Close R"] = (yahoo_minute_R["Close"]-yahoo_minute_R["Entry"])/(yahoo_minute_R["Entry"]-yahoo_minute_R["SL"])
yahoo_minute_R["Open R"] = (yahoo_minute_R["Open"]-yahoo_minute_R["Entry"])/(yahoo_minute_R["Entry"]-yahoo_minute_R["SL"])

# merge all of the mentioned above to a fully merged dataset
df_merged = yahoo_minute_R.merge(selected_spy_minute, on = ["Date","Time"],how = "left")
df_merged=df_merged.round(3)


### Critical Data

In this section we will add additional features extracted from the data which may have an impact on the outcome of a given trade.



In [21]:
critical_data = core_data[["Symbol","Num","Date"]].copy()

##### Get the week and month  SPY and symbol percent change
This feature is ment to give a global perspective on the general market bias and the given stocks bias (bullish/bearish).


In [22]:
spy_d1

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,RV,Gap
0,1993-01-29,43.968750,43.968750,43.750000,43.937500,25.799770,1003200,SPY,1.0,None
1,1993-02-01,43.968750,44.250000,43.968750,44.250000,25.983273,480500,SPY,0.648,0.071124
2,1993-02-02,44.218750,44.375000,44.125000,44.343750,26.038315,201300,SPY,0.358,-0.070621
3,1993-02-03,44.406250,44.843750,44.375000,44.812500,26.313566,529400,SPY,0.956,0.140944
4,1993-02-04,44.968750,45.093750,44.468750,45.000000,26.423655,531500,SPY,0.968,0.348675
...,...,...,...,...,...,...,...,...,...,...
7166,2021-07-15,434.809998,435.529999,432.720001,434.750000,434.750000,55126400,SPY,0.818,-0.3278
7167,2021-07-16,436.010010,436.059998,430.920013,431.339996,431.339996,75784700,SPY,1.119,0.289824
7168,2021-07-19,426.190002,431.410004,421.970001,424.970001,424.970001,147987000,SPY,2.145,-1.193952
7169,2021-07-20,425.679993,432.420013,424.829987,431.059998,431.059998,99501600,SPY,1.43,0.167069


In [23]:
# extract D1 percent change

def percentChange(symbol,date,period,data):
    
    first = []
    while len(first)==0:
        per_delta = datetime.timedelta(days = period)
        first = data[(data["Date"]==(date-per_delta)) & (data["Symbol"]==symbol)]
        period +=1

    first_open = first["Open"].values[0]   
    last = data[(data["Date"]==date) & (data["Symbol"]==symbol)]  

    last_open = last["Open"].values[0]  


    change = last_open-first_open

    percent_change = (change/first_open)*100
    return percent_change
    
week = 7
month = 30
new_critical_data = critical_data.copy()

new_critical_data["Week Change"] = critical_data.apply(lambda x: percentChange(
                                                               symbol =x["Symbol"] ,
                                                               date = x["Date"],
                                                               period = week,
                                                               data  = yahoo_d1),                                                               
                                                               axis=1)  
                                                       
new_critical_data["Month Change"] = critical_data.apply(lambda x: percentChange(
                                                               symbol =x["Symbol"] ,
                                                               date = x["Date"],
                                                               period = month,
                                                               data  = yahoo_d1),                                                               
                                                               axis=1)
                                                        
new_critical_data["spy_Week Change"] = critical_data.apply(lambda x: percentChange(
                                                               symbol ="SPY" ,
                                                               date = x["Date"],
                                                               period = week,
                                                               data  = spy_d1),                                                               
                                                               axis=1) 
                                                        
                                                        
new_critical_data["spy_Month Change"] = critical_data.apply(lambda x: percentChange(
                                                               symbol ="SPY" ,
                                                               date = x["Date"],
                                                               period = month,
                                                               data  = spy_d1),                                                               
                                                               axis=1)
# length warning
if len(critical_data)!=len(new_critical_data):
    print("**** WARNING! the gap_critical_data data and critical data lenths do not match! **** ")
critical_data=new_critical_data

##### Get Symbol Gap Percent Change

In [24]:
gap_critical_data = critical_data.merge(yahoo_d1[["Gap","Symbol","Date"]],on = ["Date","Symbol"],how = "inner")
gap_critical_data = gap_critical_data.drop_duplicates(subset = ["Date","Symbol","Num"])

# length warning
if len(critical_data)!=len(gap_critical_data):
    print("**** WARNING! the gap_critical_data data and critical data lenths do not match! **** ")
critical_data=gap_critical_data

##### Get SPY Gap Percent Change

In [25]:
spy_d1_copy = spy_d1.copy()

spy_d1_copy.rename(columns = {'Gap':'spy_Gap'}, inplace = True)

gap_spy_critical_data = critical_data.merge(spy_d1_copy[["spy_Gap","Date"]],on = ["Date"],how = "left")





# length warning
if len(critical_data)!=len(gap_spy_critical_data):
    print("**** WARNING! the gap_critical_data data and critical data lenths do not match! **** ")
critical_data=gap_spy_critical_data

##### Sum Volume at Entry 

The sum the volume from market open at 9:30 until the trade entry of a given symbol.

In [26]:
#volume at entry
q1 = df_merged[
    (df_merged["Entry Time"]>=df_merged["Time"]) ].copy()

q1["Sum Volume at Entry"] = q1["Volume"]

q2 = q1[["Num","Sum Volume at Entry"]].groupby("Num").sum().reset_index()



new_critical_data = critical_data.merge(q2, on ="Num",how= "inner")

if len(new_critical_data )!=len(critical_data):
    print("********* WARNING! critical data length is not the same after  the changes 1********")
critical_data=new_critical_data

##### Get the 66 days relative volume at entry time 

In [27]:
temp_d1 = yahoo_d1.copy()

temp_d1["RV 66"]=temp_d1["RV"]
new_critical_data = critical_data.merge(temp_d1[["Date","Symbol","RV 66"]],on = ["Date","Symbol"])
new_critical_data = new_critical_data.drop_duplicates(subset = ["Date","Symbol","Num"])



if len(new_critical_data )!=len(critical_data):
    print("********* WARNING! critical data length is not the same after the changes *********")
critical_data = new_critical_data 

#### Entry relative distance to VWAP

In [28]:
#volume at entry
q1 = df_merged[
    (df_merged["Entry Time"]>=df_merged["Time"]) ].copy()

q2 = q1.groupby(by="Num").min()

q2 = q2[["Low"]].reset_index()

q3 = df_merged[
    (df_merged["Entry Time"]==df_merged["Time"]) ].copy()

q3=q3[["Num","VWAP","Entry"]].merge(q2,on = ["Num"])
q3["VWAP rd"] =(q3["Entry"]-q3["Low"])/ (q3["VWAP"]-q3["Low"])

new_critical_data = critical_data.merge(q3[["Num","VWAP rd"]], on = ["Num"])

if len(new_critical_data )!=len(critical_data):
    print("********* WARNING! critical data length is not the same after  the changes 1********")

critical_data=new_critical_data

##### Change critical data to derivatives, store it for the main project.

In [29]:
critical_data.to_excel('Derivatives.xlsx',index = False)

ERROR -1 2103 Market data farm connection is broken:hfarm
ERROR -1 2103 Market data farm connection is broken:hfarm
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2103 Market data farm connection is broken:hfarm
ERROR -1 2103 Market data farm connection is broken:hfarm
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2103 Market data farm connection is broken:cashfarm
ERROR -1 2103 Market da

ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is

ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1102 Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; hfarm; usfuture; jfarm; eufarm; cashfarm; usfarm; euhmds; ushmds.nj; fundfarm; ushmds; secdefeu.
ERROR -1 1102 Connectivity between IB and Trader Workstation has been restored -